Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [14]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [15]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [16]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.0001

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
%%time

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.141737
Training accuracy: 10.5%
Validation accuracy: 11.1%
Loss at step 100: 2.599640
Training accuracy: 72.1%
Validation accuracy: 70.7%
Loss at step 200: 2.140964
Training accuracy: 75.0%
Validation accuracy: 72.7%
Loss at step 300: 1.877906
Training accuracy: 76.3%
Validation accuracy: 73.7%
Loss at step 400: 1.693427
Training accuracy: 77.2%
Validation accuracy: 74.3%
Loss at step 500: 1.554527
Training accuracy: 78.0%
Validation accuracy: 74.7%
Loss at step 600: 1.445027
Training accuracy: 78.7%
Validation accuracy: 74.9%
Loss at step 700: 1.355418
Training accuracy: 79.2%
Validation accuracy: 75.0%
Loss at step 800: 1.280187
Training accuracy: 79.8%
Validation accuracy: 75.3%
Test accuracy: 82.6%
Wall time: 1min 59s


In [9]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  relu_size = image_size * image_size * 2

  weights_into_relu = tf.Variable(tf.truncated_normal([image_size * image_size, relu_size]))
  weights_out_relu  = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
  biases_into_relu = tf.Variable(tf.zeros([relu_size]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases
      

  beta = tf.Variable(tf.truncated_normal([1]))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    0.00001*tf.nn.l2_loss(weights_out_relu))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)

In [10]:
%%time

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 359.407837
Minibatch accuracy: 10.9%
Validation accuracy: 29.0%
Minibatch loss at step 500: 27.766539
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 9.364817
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 13.101094
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 4.137550
Minibatch accuracy: 89.8%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 5.645955
Minibatch accuracy: 82.8%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 3.811136
Minibatch accuracy: 79.7%
Validation accuracy: 82.9%
Test accuracy: 89.8%
Wall time: 13.6 s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
beta = 0

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [25]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.950157
Training accuracy: 7.0%
Validation accuracy: 15.6%
Loss at step 100: 0.021918
Training accuracy: 100.0%
Validation accuracy: 46.9%
Loss at step 200: 0.008711
Training accuracy: 100.0%
Validation accuracy: 47.8%
Loss at step 300: 0.005511
Training accuracy: 100.0%
Validation accuracy: 48.4%
Loss at step 400: 0.004054
Training accuracy: 100.0%
Validation accuracy: 48.9%
Loss at step 500: 0.003215
Training accuracy: 100.0%
Validation accuracy: 49.2%
Loss at step 600: 0.002669
Training accuracy: 100.0%
Validation accuracy: 49.4%
Loss at step 700: 0.002283
Training accuracy: 100.0%
Validation accuracy: 49.7%
Loss at step 800: 0.001997
Training accuracy: 100.0%
Validation accuracy: 49.8%
Test accuracy: 54.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [45]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  relu_size = image_size * image_size * 2

  weights_into_relu = tf.Variable(tf.truncated_normal([image_size * image_size, relu_size]))
  weights_out_relu  = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
  biases_into_relu = tf.Variable(tf.zeros([relu_size]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
#   logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu),
#                                    0.8), 
#                      weights_out_relu) + biases
  logits_train = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu),
                     tf.nn.dropout(weights_out_relu, 0.5)) + biases
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu),
                     weights_out_relu) + biases
      

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_train) +
    beta*tf.nn.l2_loss(weights_out_relu))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)

In [48]:
len(train_labels)

200000

In [46]:
%%time
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 629.177612
Minibatch accuracy: 14.1%
Validation accuracy: 30.1%
Minibatch loss at step 500: 74.440979
Minibatch accuracy: 82.8%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 34.806770
Minibatch accuracy: 78.9%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 21.970699
Minibatch accuracy: 85.2%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 12.155595
Minibatch accuracy: 83.6%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 9.504219
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 6.364718
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Test accuracy: 87.0%
Wall time: 49.6 s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [179]:
# Minibatch loss at step 3000: 5.147270
# Minibatch accuracy: 71.1%
# Validation accuracy: 76.2%
# Test accuracy: 83.6%

batch_size = 128
beta = 0.0001
relu_size = image_size * image_size * 2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_relu0 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_size]))
  biases_relu0 = tf.Variable(tf.zeros([relu_size]))
  
  weights_relu1 = tf.Variable(tf.truncated_normal([relu_size, relu_size]))
  biases_relu1 = tf.Variable(tf.zeros([relu_size]))

  weights_reluN  = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
  biases_reluN = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.

  logits_t = tf.nn.relu6(tf.matmul(tf_train_dataset, weights_relu0) + biases_relu0)
  logits_t = tf.nn.dropout(tf.nn.relu6(tf.matmul(logits_t, weights_relu1) + biases_relu1), 0.5)
  logits_t = tf.matmul(logits_t, weights_reluN) + biases_reluN
    
  logits = tf.nn.relu6(tf.matmul(tf_train_dataset, weights_relu0) + biases_relu0)
  logits = tf.nn.relu6(tf.matmul(logits, weights_relu1) + biases_relu1)
  logits = tf.matmul(logits, weights_reluN) + biases_reluN

      

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_t)
#     + beta * tf.nn.l2_loss(weights_reluN)
    )
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_rate = 0.5
  decay_steps = 500
  learning_rate = tf.train.exponential_decay(decay_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.relu6(tf.matmul(tf_valid_dataset, weights_relu0) + biases_relu0)
  valid_prediction = tf.nn.relu6(tf.matmul(valid_prediction, weights_relu1) + biases_relu1)
  valid_prediction = tf.matmul(valid_prediction, weights_reluN) + biases_reluN
  valid_prediction = tf.nn.softmax(valid_prediction)
  

  test_prediction = tf.nn.relu6(tf.matmul(tf_test_dataset, weights_relu0) + biases_relu0)
  test_prediction = tf.nn.relu6(tf.matmul(test_prediction, weights_relu1) + biases_relu1)
  test_prediction = tf.matmul(test_prediction,weights_reluN) + biases_reluN
  test_prediction = tf.nn.softmax(test_prediction)

In [53]:
# Minibatch loss at step 3000: 1.184085
# Minibatch accuracy: 72.7%
# Validation accuracy: 75.2%
# Test accuracy: 82.9%

batch_size = 128
beta = 0.0001
relu_size = image_size * image_size

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_relu0 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_size]))
  biases_relu0 = tf.Variable(tf.zeros([relu_size]))
  
  weights_relu1 = tf.Variable(tf.truncated_normal([relu_size, relu_size]))
  biases_relu1 = tf.Variable(tf.zeros([relu_size]))

  weights_reluN  = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
  biases_reluN = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.

  logits_t = tf.nn.relu6(tf.matmul(tf_train_dataset, weights_relu0) + biases_relu0)
  logits_t = tf.nn.dropout(tf.nn.relu6(tf.matmul(logits_t, weights_relu1) + biases_relu1), 0.5)
  logits_t = tf.matmul(logits_t, weights_reluN) + biases_reluN
    
  logits = tf.nn.relu6(tf.matmul(tf_train_dataset, weights_relu0) + biases_relu0)
  logits = tf.nn.relu6(tf.matmul(logits, weights_relu1) + biases_relu1)
  logits = tf.matmul(logits, weights_reluN) + biases_reluN

      

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_t)
#     + beta * tf.nn.l2_loss(weights_reluN)
    )
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_rate = 0.5
  decay_steps = 500
  learning_rate = tf.train.exponential_decay(decay_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.relu6(tf.matmul(tf_valid_dataset, weights_relu0) + biases_relu0)
  valid_prediction = tf.nn.relu6(tf.matmul(valid_prediction, weights_relu1) + biases_relu1)
  valid_prediction = tf.matmul(valid_prediction, weights_reluN) + biases_reluN
  valid_prediction = tf.nn.softmax(valid_prediction)
  

  test_prediction = tf.nn.relu6(tf.matmul(tf_test_dataset, weights_relu0) + biases_relu0)
  test_prediction = tf.nn.relu6(tf.matmul(test_prediction, weights_relu1) + biases_relu1)
  test_prediction = tf.matmul(test_prediction,weights_reluN) + biases_reluN
  test_prediction = tf.nn.softmax(test_prediction)

In [69]:
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512
num_hidden_nodes3 = 256
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
  weights4 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes3, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes3)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  drop1 = tf.nn.dropout(lay1_train, 0.5)
  lay2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
  drop2 = tf.nn.dropout(lay2_train, 0.5)
  lay3_train = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
  drop3 = tf.nn.dropout(lay3_train, 0.5)
  logits = tf.matmul(drop3, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
  lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
  lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(lay3_test, weights4) + biases4)

In [106]:
batch_size = 1024
beta = [0.2, 0.05, 0.01, 0.002, 0.0004]
relu_size = [image_size * image_size, 1024, 512, 256, 128, num_labels]
layers_count = len(relu_size) - 1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  weights_relu = []
  biases_relu = []
  
  for i in range(layers_count):
        weights_relu.append(tf.Variable(tf.truncated_normal([relu_size[i],relu_size[i+1]], 
                                                            stddev=np.sqrt(2.0 / relu_size[i]))))
        biases_relu.append(tf.Variable(tf.zeros([relu_size[i+1]])))

  
  # Training computation.

#   logits_t = tf.matmul(tf_train_dataset, weights_relu0)
    
  logits_t = tf_train_dataset
  for i in range(layers_count-1):
    logits_t = tf.nn.relu(tf.nn.dropout(tf.matmul(logits_t, weights_relu[i]) + biases_relu[i], 0.5))
  logits_t = tf.matmul(logits_t, weights_relu[layers_count-1]) + biases_relu[layers_count-1]
    
  logits = tf_train_dataset
  for i in range(layers_count-1):
    logits = tf.nn.relu(tf.matmul(logits, weights_relu[i]) + biases_relu[i])
  logits = tf.matmul(logits, weights_relu[layers_count-1]) + biases_relu[layers_count-1]

  loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_t)
  for i in range(layers_count-1):
    lost = tf.add(loss, beta[i] * tf.nn.l2_loss(weights_relu[i]))
    
  loss = tf.reduce_mean(loss)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_rate = 0.95
  decay_steps = 200
  learning_rate = tf.train.exponential_decay(decay_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf_valid_dataset
  for i in range(layers_count-1):
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, weights_relu[i]) + biases_relu[i])
  valid_prediction = tf.matmul(valid_prediction, weights_relu[layers_count-1]) + biases_relu[layers_count-1]
  valid_prediction = tf.nn.softmax(valid_prediction)
  

  test_prediction = tf_test_dataset
  for i in range(layers_count-1):
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, weights_relu[i]) + biases_relu[i])
  test_prediction = tf.matmul(test_prediction, weights_relu[layers_count-1]) + biases_relu[layers_count-1]
  test_prediction = tf.nn.softmax(test_prediction)

In [107]:
%%time
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.943242
Minibatch accuracy: 12.1%
Validation accuracy: 14.1%
Minibatch loss at step 500: 0.542684
Minibatch accuracy: 86.1%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.469448
Minibatch accuracy: 89.2%
Validation accuracy: 88.0%
Minibatch loss at step 1500: 0.398835
Minibatch accuracy: 89.4%
Validation accuracy: 88.6%
Minibatch loss at step 2000: 0.433108
Minibatch accuracy: 88.9%
Validation accuracy: 89.1%
Minibatch loss at step 2500: 0.387615
Minibatch accuracy: 89.8%
Validation accuracy: 89.6%
Minibatch loss at step 3000: 0.357869
Minibatch accuracy: 91.3%
Validation accuracy: 89.8%
Minibatch loss at step 3500: 0.330010
Minibatch accuracy: 91.5%
Validation accuracy: 90.1%
Minibatch loss at step 4000: 0.293007
Minibatch accuracy: 93.5%
Validation accuracy: 90.1%
Minibatch loss at step 4500: 0.319398
Minibatch accuracy: 91.7%
Validation accuracy: 90.4%
Minibatch loss at step 5000: 0.319424
Minibatch accuracy: 92.4%
Validation accurac